<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|

`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|

`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [45]:
-- What is the oldest business on each continent?
WITH row_assigned AS(
	SELECT
		business,
		continent,
		country,
		year_founded,
		--Ranks businesses within each continent based on their founding year, with the oldest receiving rank 1
		ROW_NUMBER() OVER(PARTITION BY continent ORDER BY year_founded ASC) AS rank
	FROM businesses b INNER JOIN countries c 
	ON b.country_code = c.country_code
	)
SELECT continent, country, business, year_founded
FROM row_assigned 
WHERE rank = 1
ORDER BY continent ASC

,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,Oceania,Australia,Australia Post,1809
5,South America,Peru,Casa Nacional de Moneda,1565


In [46]:
-- How many countries per continent lack data on the oldest businesses
SELECT
	continent,
	COUNT(*)
FROM businesses b RIGHT JOIN countries c 
ON b.country_code = c.country_code
--Filter out columns that lack business data
WHERE business IS NULL
GROUP BY continent
ORDER BY continent ASC

,continent,count
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,6
4,Oceania,11
5,South America,3


In [47]:
-- How many countries per continent lack data on the oldest businesses
-- Does including the `new_businesses` data change the previous results?
SELECT c.continent, COUNT(c.country_code) AS countries_without_businesses
FROM countries c
LEFT JOIN (
    SELECT * FROM businesses
    --Combine the 2 tables
	UNION ALL
    SELECT * FROM new_businesses
) b
ON c.country_code = b.country_code
--Filter out columns that lack business data
WHERE b.business IS NULL
GROUP BY c.continent

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [48]:
-- Which business categories are best suited to last over the course of centuries?
WITH row_assigned AS(
	SELECT
		continent,
		category,
		year_founded,
		--Ranks businesses within each continent-category combination based on their founding year
		ROW_NUMBER() OVER(PARTITION BY continent, category ORDER BY year_founded ASC) AS rank
	FROM businesses b INNER JOIN countries c
	ON b.country_code = c.country_code
	INNER JOIN categories cat ON b.category_code = cat.category_code
	)
SELECT continent, category, year_founded
FROM row_assigned 
-- Filters for the oldest founding year in each continent-category combination.
WHERE rank = 1
ORDER BY year_founded ASC

,continent,category,year_founded
0,Asia,Construction,578
1,Europe,"Cafés, Restaurants & Bars",803
2,Europe,"Distillers, Vintners, & Breweries",862
3,Europe,Manufacturing & Production,864
4,Asia,"Cafés, Restaurants & Bars",1153
5,Europe,Agriculture,1218
6,Europe,Tourism & Hotels,1230
7,Europe,Mining,1248
8,Europe,Medical,1422
9,Europe,Postal Service,1520
